In [2]:
!pip install vk_api

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [5]:
import re
import pandas as pd
import networkx as nx

In [14]:
import pandas as pd
from pandas import read_csv

In [3]:
import vk_api
vk_session = vk_api.VkApi('телефон', 'пароль')
vk_session.auth()

vk = vk_session.get_api()

In [9]:
# Это нужно для построения графа и визуализации, а также стемминга
# import matplotlib.pyplot as plt
# from matplotlib import style
# from tqdm import tqdm
# from pymystem3 import Mystem
# import sqlite3

In [4]:
vk.wall.get(domain="freefinnougria", count=100)

{'count': 6762,
 'items': [{'attachments': [{'photo': {'access_key': 'fc1314ee0df7358cd6',
      'album_id': -7,
      'date': 1498326253,
      'id': 456240219,
      'owner_id': -104640911,
      'sizes': [{'height': 107,
        'type': 'm',
        'url': 'https://sun9-66.userapi.com/c837623/v837623984/486f7/sX-0UcCySuo.jpg',
        'width': 130},
       {'height': 107,
        'type': 'o',
        'url': 'https://sun9-44.userapi.com/c837623/v837623984/486fa/0x5fezDzl8s.jpg',
        'width': 130},
       {'height': 165,
        'type': 'p',
        'url': 'https://sun9-51.userapi.com/c837623/v837623984/486fb/4GLSmI6VuYQ.jpg',
        'width': 200},
       {'height': 264,
        'type': 'q',
        'url': 'https://sun9-2.userapi.com/c837623/v837623984/486fc/ry9CE_e1aHg.jpg',
        'width': 320},
       {'height': 421,
        'type': 'r',
        'url': 'https://sun9-60.userapi.com/c837623/v837623984/486fd/UYQTZPBMIqg.jpg',
        'width': 510},
       {'height': 62,
        

In [6]:
# d = {}
# a = vk.wall.getComments(owner_id = '-104640911', post_id = '16036', comment_id = 16037, 
#                     count = 100, v = '5.91').get('items')


In [31]:
# Это класс, который достает информацию о комментарии 
#(функция get_speaker возвращает имя комментатора
# функция get_whom возвращает имя комментируемого
# функция get_text возвращает текст комментария
# функция answer возвращает лист вида ["кто", "кому"] для построения графа)
# функция get_last_id возвращает id пользователя, чей комментарий был только что обработан
class Comment:
    
    def __init__(self, i):
        self.speaker = None
        self.text = None
        self.whom = None
        self.i = i
        
    def get_speaker(self):
        name = self.i.get('from_id')
        return name
    
    def get_whom(self):
        text = self.i.get('text')
        pattern = re.compile('\[id\d+|\w+\]')
        whom = pattern.findall(text)
        whom = [wh.strip(']').strip('[') for wh in whom]
        whom = self.i.get('reply_to_user')
#         print(lol)
        return str(whom)
        
    def get_text(self):
        text = self.i.get('text')
        pattern = re.compile('\[id\d+\|\w+\]')
        text_del = pattern.findall(text)
        text_del = str(text_del).replace("'[", '').replace("]'", '')
#         print(text_del)
        text =  text.replace((text_del), '')
#         print(text)
        return text
    
    def answer(self, who, whom, text): 
        list_who_to_whom = []
        list_who_to_whom.append(who)
        list_who_to_whom.append(text)
        try:
            list_who_to_whom.append(whom)
        except TypeError:
            list_who_to_whom.append('')
        return list_who_to_whom
    
    def get_last_id(self):
        last_id = self.i.get('id')
        print(last_id)
        return last_id

In [32]:
#Код последовательно проходится по всем комментам и выкачивает инфу о них
list_speakers = []
list_texts = []
list_whom = []
list_answers = []
last_id =  16040
count = 0
while count != 3:
    a = vk.wall.getComments(owner_id = '-104640911', post_id = '16036', comment_id = 16037,  
                    start_comment_id = last_id, count = 100, v = '5.91').get('items')
    for i in a:
        com = Comment(i)
        who = com.get_speaker()
        whom = com.get_whom()
        list_speakers.append(who)
        text = com.get_text()
        list_texts.append(text)
        list_whom.append(str(whom))
        list_answers.append(com.answer(who, whom, text))
        com2 = Comment(i)
    last_id = com2.get_last_id()
    count += 1


    
# print(list_speakers)
# print(list_texts)
# print(list_whom)
# print(list_answers)

16153
16271
24614


In [33]:
#эта часть кода создает граф (недоработано)
# G = nx.Graph()
# G.add_nodes_from(list_speakers)
# G.add_edge(1, 3, weight=4)
# G.add_edges_from(list_answers)

In [34]:
#эта часть кода создает граф (недоработано)
# style.use('ggplot')
# pos=nx.spring_layout(G)
# nx.draw_networkx_nodes(G, pos, node_color='red', node_size=10) 
# nx.draw_networkx_edges(G, pos, edge_color='yellow')
# nx.draw_networkx_labels(G, pos, font_size=20, font_family='Arial')
# plt.axis('off') 
# plt.show()

In [35]:
# list_speakers_not_repeat - лист в котором записаны все id пользователей, участвующие в диалоге
list_speakers_not_repeat = []
for speaker in list_speakers: 
    if speaker not in list_speakers_not_repeat:
        list_speakers_not_repeat.append(speaker)
print(list_speakers_not_repeat)

[171093523, -104640911, 18374082, 534347283, 7324384, 2305053, 209776266, 517048608, 356600647, 444895706, 136531491, 231456575, 29235862, 169944768, 279825064, 226600262, 9352436, 178146854]


In [36]:
# list_whom_not_repeat = []
# print()
# for i in list_whom:
#     if i not in list_whom_not_repeat and i != []:
#         i[0] = i[0].lstrip('id')
#         list_whom_not_repeat.append(i)
#         print(i)
# print(list_whom_not_repeat)

In [37]:
# named_speakers = []
# for i in list_whom_not_repeat:
#     named_speakers.append(i[1])
# G.add_nodes_from(named_speakers)
# G.add_edges_from(list_answers)

In [38]:
# list_whom = list(map(str, list_whom))

In [39]:
# создает и записывает в табличку output.xlsx данные 
d = {'имя коментатора': list_speakers, 'текст комментария': list_texts, 'имя комментируемого': list_whom}
df = pd.DataFrame(data=d)
df.to_excel("output.xlsx")

In [40]:
df

,имя коментатора,имя комментируемого,текст комментария
0,171093523,None,", конечно же вы не финно-угр, или чухна, как г..."
1,-104640911,None,", 80\20 таких соотношений нет.\nБелое населени..."
2,18374082,171093523,язык может не знать карел.потому что есть семь...
3,171093523,18374082,", если б хотели, выучили. Сейчас никто не запр..."
4,18374082,171093523,", учу финский."
5,171093523,18374082,", если финский - язык ваших предков, то это от..."
6,18374082,171093523,", нет,карелка я."
7,171093523,18374082,", ... а карельский почему не учите? Не нужен?"
8,534347283,None,", Русские образовались на 80% из финно-угорски..."
9,171093523,534347283,"Если он не хочет быть карелом, не надо заставл..."
